In [4]:
# Importando bibliotecas
from functions import *
import pandas as pd
import locale
from pathlib import Path
import shutil
from datetime import datetime
import warnings
import logging
from openpyxl import load_workbook

timer = Temporizador()
timer.iniciar()

locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.expand_frame_repr', False)

# Detecta se o script está sendo executado de um .py ou de um notebook
try:
    caminho_base = Path(__file__).resolve().parent
except NameError:
    # __file__ não existe em Jupyter ou ambiente interativo
    caminho_base = Path.cwd()


pasta_input_parquet = caminho_base.parent / '01_INPUT_PIPELINE/01_BD_PARQUET'
pasta_staging_parquet = caminho_base.parent / '02_STAGING_PARQUET'
pasta_painel = caminho_base.parent / '05_PAINEL'
pasta_historico_planos = caminho_base.parent / '04_HISTORICO_PLANOS'

print("✅ Mapeamento de pastas concluído com sucesso!")

✅ Mapeamento de pastas concluído com sucesso!


In [ ]:
#ler parquet df_vendas_krona
df_vendas_krona = pd.read_parquet(pasta_staging_parquet / 'df_vendas_krona.parquet')

# Filtrar df_vendas_krona ultimos 12 meses
df_vendas_krona_12m = df_vendas_krona[df_vendas_krona['PERIODO'] >= (datetime.now() - pd.DateOffset(months=12))]

# aGREGAR df_vendas_krona_12m por EMPRESA, COD_PROD, DESC_PRODUTO, FAMILIA, LINHA, REGIONAL_GESTOR, REGIONAL, PERIODO, SOMANDO QTD_VENDA E VOL_VENDA
df_vendas_krona_12m_agrupado = df_vendas_krona_12m.groupby(['COD_PROD', 'DESC_PRODUTO', 'FAMILIA', 'LINHA', 'REGIONAL_GESTOR', 'REGIONAL']).agg({'QTD_VENDA': 'sum', 'VOL_VENDA': 'sum'}).reset_index()

# Eliminar linhas com QTD_VENDA
df_vendas_krona_12m_agrupado = df_vendas_krona_12m_agrupado[df_vendas_krona_12m_agrupado['QTD_VENDA'] > 0]

# # Criar coluna divindo VOL_VENDA por QTD_VENDA, nome como PESO_UNIT
df_vendas_krona_12m_agrupado['PESO_UNIT'] = df_vendas_krona_12m_agrupado['VOL_VENDA'] / df_vendas_krona_12m_agrupado['QTD_VENDA']

# Criar DIM_PRODUTO com COD_PROD, DESC_PROD, PESO_UNIT
df_dim_produto = (
    df_vendas_krona_12m_agrupado
    .drop_duplicates(subset=['COD_PROD'])
    [['COD_PROD', 'DESC_PRODUTO', 'PESO_UNIT']]
    .reset_index(drop=True)
)            

# # Remover colunas QTD_VENDA e PESO_UNIT do df_vendas_krona_12m_agrupado
df_vendas_krona_12m_agrupado = df_vendas_krona_12m_agrupado.drop(columns=['QTD_VENDA', 'PESO_UNIT'])

# Criar uma coluna TOTAL_VOL_VENDA, somando por REGIONAL_GESTOR, REGIONAL, FAMILIA
df_vendas_krona_12m_agrupado['TOTAL_VOL_VENDA'] = df_vendas_krona_12m_agrupado.groupby(['REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA'])['VOL_VENDA'].transform('sum')

# Criar coluna PERC_PART, dividindo VOL_VENDA por TOTAL_VOL_VENDA
df_vendas_krona_12m_agrupado['PERC_PART'] = df_vendas_krona_12m_agrupado['VOL_VENDA'] / df_vendas_krona_12m_agrupado['TOTAL_VOL_VENDA']

# ler arquivo PLANO_REGIONAL.csv da pasta 04_HISTORICO_PLANOS
df_plano_regional = pd.read_csv(
    pasta_historico_planos / 'PLANO_REGIONAL.csv',
    sep=';',
    dtype=str,          # força tudo texto
    encoding='latin1'   # evita dor de cabeça
)

df_plano_regional['VALOR'] = (
    df_plano_regional['VALOR']
    .str.replace('.', '', regex=False)   # se existir milhar
    .str.replace(',', '.', regex=False)  # vírgula → ponto
    .astype(float)
)

# Eliminar colunas ID, TIPO, ORIGEM_PLANO, VERSAO_PLANO
df_plano_regional = df_plano_regional.drop(columns=['ID', 'TIPO', 'ORIGEM_PLANO', 'VERSAO_PLANO'])


# Filtrar alguns dados para validar, como por exemplo, REGIONAL_GESTOR = 'CO NO' e FAMILIA = '00 - TUBOS PVC KRONA'
df_plano_regional_filtrado = df_plano_regional[
    (df_plano_regional['REGIONAL_GESTOR'] == 'CO NO') & 
    (df_plano_regional['REGIONAL'] == 'CENTRO OESTE') & 
    (df_plano_regional['FAMILIA'] == '00 - TUBOS PVC KRONA')
]

df_plano_explodido = df_plano_regional.merge(
    df_vendas_krona_12m_agrupado[
        ['COD_PROD', 'DESC_PRODUTO', 'FAMILIA', 'LINHA', 
         'REGIONAL_GESTOR', 'REGIONAL', 'PERC_PART']
    ],
    on=['REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA'],
    how='left'
)

df_plano_explodido['KG_SKU'] = (
    df_plano_explodido['VALOR'] *
    df_plano_explodido['PERC_PART']
)


df_plano_explodido = df_plano_explodido.merge(
    df_dim_produto[['COD_PROD', 'PESO_UNIT']],
    on=['COD_PROD'],
    how='left'
)


df_plano_explodido['QTD_SKU'] = (
    df_plano_explodido['KG_SKU'] /
    df_plano_explodido['PESO_UNIT']
)

# Eliminar PERIODOS que a soma das linhas deles for 0
periodos_com_soma_zero = df_plano_explodido.groupby('PERIODO')['KG_SKU'].sum()
periodos_com_soma_zero = periodos_com_soma_zero[periodos_com_soma_zero == 0].index
df_plano_explodido = df_plano_explodido[~df_plano_explodido['PERIODO'].isin(periodos_com_soma_zero)]

# Salvar na minha DESKTOP um arquivo Excel com o nome PLANO_EXPLODIDO.xlsx, sem index, e com a formatação de número com vírgula e ponto decimal
df_plano_explodido.to_excel(
    r"C:\Users\carlo\OneDrive\Desktop\PLANO_EXPLODIDO.xlsx",
    index=False,
    float_format="%.2f"
)

# # Filtrar alguns dados para validar, como por exemplo, REGIONAL_GESTOR = 'CO NO' e FAMILIA = '00 - TUBOS PVC KRONA'
# df_plano_explodido_filtrado = df_plano_explodido[
#     (df_plano_explodido['REGIONAL_GESTOR'] == 'CO NO') & 
#     (df_plano_explodido['REGIONAL'] == 'CENTRO OESTE') & 
#     (df_plano_explodido['FAMILIA'] == '00 - TUBOS PVC KRONA') &
#     (df_plano_explodido['COD_PROD'] == '0023') &
#     (df_plano_explodido['PERIODO'] == '2026-03-01')
# ].reset_index(drop=True)

# df_plano_explodido_filtrado
